In [1]:
# !pip install pandas tqdm transformers accelerate bitsandbytes

In [2]:
import re
import os
import pandas as pd
from tqdm import tqdm

import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [3]:
test = pd.read_csv('../data/test.csv')
test

,ID,Question
0,TEST_000,금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융...
1,TEST_001,위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?\n1 수행인력\n...
2,TEST_002,관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?\n...
3,TEST_003,재해 복구 계획 수립 시 고려해야 할 요소로 옳지 않은 것은?\n1 복구 절차 수립...
4,TEST_004,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표...
...,...,...
510,TEST_510,"""정보보호최고책임자""의 임명에 관한 설명으로 옳지 않은 것은?\n1 정보보호최고책임..."
511,TEST_511,IPv6 주소 체계의 주요 특징으로 옳지 않은 것은?\n1 NAT 필요성 감소\n2...
512,TEST_512,하이브리드 위협에 대한 설명으로 가장 적절한 것은?\n1 사이버 공간에서만 발생하는...
513,TEST_513,전자금융거래법의 주요 목적 중 하나는 무엇인가?\n1 전자금융거래의 비대면성 강화\...


In [4]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    """
    객관식 여부를 판단: 2개 이상의 숫자 선택지가 줄 단위로 존재할 경우 객관식으로 간주
    """
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2


# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    """
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())

    question = " ".join(q_lines)
    return question, options

In [5]:
# 프롬프트 생성기
def make_prompt_auto(text):
    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                # "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                "아래 질문에 대해 정답의 핵심 키워드와 의미를 모두 포함하여 3문장 이내로 간결하게 답변하세요. 군더더기 없이 요점만 명확하게 작성하세요.\n\n"
                f"질문: {text}\n\n"
                "답변:"
                )
    return prompt

In [6]:
# -*- coding: utf-8 -*-
# 멀티법령 RAG 전처리/인덱싱 (PIPA, 신용정보법, 전자서명법, 정보통신망법)

import os, re, json, math
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceEmbeddings
import faiss

# ===== 사용자 환경 상수 =====
# LLM_ID = "nlpai-lab/KULLM3"
LLM_ID = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
EMB_MODEL = "jhgan/ko-sroberta-multitask"   # 경량/호환성 위주
CHUNK_TOKENS = 600
CHUNK_OVERLAP = 32
CTX_TOKEN_BUDGET = 600
TOP_K = 4
SEED = 42
torch.manual_seed(SEED)

# ===== 법령 설정: 파일 경로 + 제거/정규식 패턴 =====
LAW_CONFIG = {
    # 개인정보 보호법
    "pipa": {
        "law_name": "개인정보 보호법",
        "pdf_path": "../data/개인정보 보호법(법률)(제19234호)(20250313).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*개인정보\s*보호법',
            r'법제처\s+\d+\s+국가법령정보센터',
            r'국가법령정보센터\s*개인정보\s*보호법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>',         # <개정 …>, <신설 …>
            r'\[[^\]]+\]',      # [본조신설 …]
        ],
    },
    # 신용정보의 이용 및 보호에 관한 법률
    "ciupa": {
        "law_name": "신용정보법",
        "pdf_path": "../data/신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*신용정보.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자서명법
    "es_act": {
        "law_name": "전자서명법",
        "pdf_path": "../data/전자서명법(법률)(제18479호)(20221020).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자서명법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 정보통신망 이용촉진 및 정보보호 등에 관한 법률
    "icn_act": {
        "law_name": "정보통신망법",
        "pdf_path": "../data/정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20678호)(20250722).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*정보통신망.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자금융거래법
    "eft_act": {
        "law_name": "전자금융거래법",
        "pdf_path": "../data/전자금융거래법(법률)(제19734호)(20240915).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자금융거래.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자금융감독규정
    "rs_act": {
        "law_name": "전자금융감독규정",
        "pdf_path": "../data/전자금융감독규정(금융위원회고시)(제2025-4호)(20250205).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자금융거래.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
}

In [7]:
# ===== 토크나이저: 토큰 길이 계산/청킹 =====
llm_tokenizer = AutoTokenizer.from_pretrained(LLM_ID)
if llm_tokenizer.pad_token is None:
    llm_tokenizer.pad_token = llm_tokenizer.eos_token
llm_tokenizer.padding_side = "right"

def token_len(s: str) -> int:
    return len(llm_tokenizer(s, add_special_tokens=False)["input_ids"])

def split_sentences_ko(text: str) -> List[str]:
    # 고정폭 lookbehind: '다.' 또는 일반 종결부호
    text = re.sub(r'\s+', ' ', text).strip()
    if not text:
        return []
    return re.split(r'(?<=다\.)\s+|(?<=[.?!。！？])\s+', text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# ===== 공통 정제 =====
def normalize_common(text: str) -> str:
    # 한자 제거
    text = re.sub(r'[\u4e00-\u9fff]', '', text)
    # circled numbers → (n)
    circled = '①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳'
    for idx, c in enumerate(circled, 1):
        text = text.replace(c, f'({idx})')
    # 공백/빈 괄호 정리
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\(\s*\)', '', text)
    return text

def clean_text_by_config(text: str, drop_patterns: List[str]) -> str:
    for pat in drop_patterns:
        text = re.sub(pat, '', text)
    return normalize_common(text)

# ===== 조문 단위 분리 =====
ARTICLE_HEADER_PATTERN = r'(제\d+조(?:의\d+)?\([^)]+\))'  # 제X조(제목) / 제X조의Y(제목)

def split_articles(raw_text: str) -> List[Tuple[str, str, str]]:
    parts = re.split(ARTICLE_HEADER_PATTERN, raw_text)
    out = []
    for i in range(1, len(parts), 2):
        header = parts[i]
        body = (parts[i+1] if i+1 < len(parts) else "").strip().replace("\n", " ")
        m = re.match(r'(제\d+조(?:의\d+)?)[(]([^)]+)[)]', header)
        if not m:
            continue
        article_id = m.group(1)          # 제xx조 / 제xx조의y
        article_title = m.group(2)       # (제목)
        out.append((article_id, article_title, body))
    return out

def chunk_article(article_body: str, header: str) -> List[str]:
    prefix = header.strip() + "\n"
    sents = split_sentences_ko(article_body) or [article_body]
    chunks, cur, cur_toks = [], [], token_len(prefix)
    for s in sents:
        tl = token_len(s)
        if cur_toks + tl > CHUNK_TOKENS and cur:
            chunks.append(prefix + " ".join(cur))
            # overlap: 마지막 문장 유지
            keep = cur[-1] if CHUNK_OVERLAP > 0 and cur else ""
            cur = [keep] if keep else []
            cur_toks = token_len(prefix) + (token_len(keep) if keep else 0)
        cur.append(s); cur_toks += tl
    if cur:
        chunks.append(prefix + " ".join(cur))
    return chunks


In [9]:
# ===== 데이터 클래스 =====
@dataclass
class LawDoc:
    text: str
    meta: Dict

# ===== 임베딩 =====
embeddings = HuggingFaceEmbeddings(
    model_name=EMB_MODEL,
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={
        "normalize_embeddings": True,
        "batch_size": 128,
        "convert_to_numpy": True,
        "convert_to_tensor": False
    }
)

# ===== 인덱스 (FAISS HNSW) =====
def build_faiss_hnsw(vectors: np.ndarray, m: int = 32, ef_search: int = 32) -> faiss.IndexHNSWFlat:
    dim = vectors.shape[1]
    idx = faiss.IndexHNSWFlat(dim, m)
    idx.hnsw.efSearch = ef_search
    idx.add(vectors.astype(np.float32))
    return idx

# ===== 파이프라인: 1) 로드 → 2) 정제 → 3) 조문분리 → 4) 청킹 → 5) 임베딩/인덱스
def load_pdf_text(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for p in reader.pages:
        t = p.extract_text() or ""
        text += t + "\n"
    return text

def preprocess_law(law_id: str, cfg: Dict) -> List[LawDoc]:
    raw = load_pdf_text(cfg["pdf_path"])
    cleaned = clean_text_by_config(raw, cfg["drop_patterns"])
    articles = split_articles(cleaned)

    docs: List[LawDoc] = []
    # 시행일 파싱(있으면): [시행 YYYY. M. D.]가 머리말에 있는 경우가 많으나 PDF마다 다름 → 필요 시 별도 파서
    effective_date = None  # 필요 시 추출 로직 추가

    for article_id, title, body in articles:
        header = f'{cfg["law_name"]} {article_id}({title})'
        chunks = chunk_article(body, header)
        for ch in chunks:
            meta = {
                "law_id": law_id,
                "law_name": cfg["law_name"],
                "article_id": article_id,
                "article_title": title,
                "effective_date": effective_date,   # None 가능
                "tok_len": token_len(ch),
                "source_uri": cfg.get("pdf_path"),
                "version": None
            }
            docs.append(LawDoc(text=ch, meta=meta))
    return docs

def build_indices(all_docs: List[LawDoc]):
    # (a) 법령별 인덱스
    per_law_docs: Dict[str, List[LawDoc]] = {}
    for d in all_docs:
        per_law_docs.setdefault(d.meta["law_id"], []).append(d)

    indices = {}
    for law_id, docs in per_law_docs.items():
        mat = np.array(embeddings.embed_documents([d.text for d in docs]), dtype=np.float32)
        indices[f"faiss_hnsw_{law_id}"] = {
            "index": build_faiss_hnsw(mat, m=32, ef_search=32),
            "docs": docs
        }

    # (b) 글로벌 인덱스
    mat_all = np.array(embeddings.embed_documents([d.text for d in all_docs]), dtype=np.float32)
    indices["faiss_hnsw_all"] = {
        "index": build_faiss_hnsw(mat_all, m=32, ef_search=32),
        "docs": all_docs
    }
    return indices

/tmp/ipykernel_1052/3198458753.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
all_docs: List[LawDoc] = []
for law_id, cfg in LAW_CONFIG.items():
    if not os.path.exists(cfg["pdf_path"]):
        print(f"[WARN] PDF not found: {cfg['pdf_path']}")
        continue
    docs = preprocess_law(law_id, cfg)
    all_docs.extend(docs)
    print(f"[OK] {cfg['law_name']} → chunks: {len(docs)}")

indices = build_indices(all_docs)
print("[OK] built indices:", list(indices.keys()))

[OK] 개인정보 보호법 → chunks: 160
[OK] 신용정보법 → chunks: 125
[OK] 전자서명법 → chunks: 26
[OK] 정보통신망법 → chunks: 113
[OK] 전자금융거래법 → chunks: 78
[OK] 전자금융감독규정 → chunks: 113
[OK] built indices: ['faiss_hnsw_pipa', 'faiss_hnsw_ciupa', 'faiss_hnsw_es_act', 'faiss_hnsw_icn_act', 'faiss_hnsw_eft_act', 'faiss_hnsw_rs_act', 'faiss_hnsw_all']


In [11]:
# ===== 라우팅 규칙 =====
ARTICLE_PTRN = re.compile(r"제\d+조(?:의\d+)?")  # 조문 표기
LAW_HINTS = {
    "pipa": ("개인정보", "개인 정보", "개인정보보호법"),
    "ciupa": ("신용정보",  "신용정보법"),
    "es_act": ("전자서명", "전자서명법"),
    "icn_act": ("정보통신망", "통신망", "정보통신망법"),
    "eft_act": ("전자금융", "금융거래", "전자금융거래법"),
    "rs_act": ("전자금융감독", "금융감독", "전자금융감독규정"),
}

def detect_law_id(query: str) -> Optional[str]:
    q = query.lower()
    for law_id, kws in LAW_HINTS.items():
        if any(kw.lower() in q for kw in kws):
            return law_id
    return None

def route_is_domain(query: str) -> bool:
    # 법/금융/보안 도메인 간단 라우터 (검색 여부 판단용)
    domain_kws = ("법", "조(", "과징금", "처벌", "보안", "침해", "금융", "개인정보", "신용정보", "전자서명", "정보통신", "전자금융", "금융감독")
    q = query.lower()
    return any(kw in q for kw in domain_kws) or bool(ARTICLE_PTRN.search(query))

def choose_index(indices: dict, query: str):
    """
    1) 질의에서 법령 단서 -> 해당 법 인덱스 우선
    2) 조문 패턴만 있거나 단서가 불분명 -> 글로벌 인덱스
    3) 아무 단서도 없으면 None (베이스모델 경로)
    """
    law_id = detect_law_id(query)
    if law_id:
        key = f"faiss_hnsw_{law_id}"
        if key in indices:
            return indices[key]  # {"index": ..., "docs": ...}
    # 법령 단서 없지만 도메인성/조문 표기는 있는 경우 글로벌
    if route_is_domain(query) and "faiss_hnsw_all" in indices:
        return indices["faiss_hnsw_all"]
    return None  # 베이스모델 직행


In [12]:
# ===== 검색 with 점수 (해당 인덱스에서) =====
def faiss_search_with_scores_from_index(index_entry: dict, query: str, top_k: int = TOP_K):
    # embeddings는 상위 스코프에서 로드되었다고 가정
    qv = np.array(embeddings.embed_query(query), dtype=np.float32).reshape(1, -1)
    D, I = index_entry["index"].search(qv, top_k)     # L2 거리 (정규화 벡터 가정)
    cos = 1.0 - (D[0] / 2.0)                          # L2 -> cosine
    out = []
    docs = index_entry["docs"]
    for idx, i in enumerate(I[0]):
        ii = int(i)
        if ii >= 0:
            out.append((docs[ii], float(cos[idx])))
    return out  # [(LawDoc, cosine), ...]

# ===== 컨텍스트 패킹 (질문 주신 코드 재사용 + tok_len 캐시) =====
def pack_context(docs_in, token_budget=CTX_TOKEN_BUDGET):
    acc, used = [], 0
    for d in docs_in:
        tl = d.meta.get("tok_len", None)
        if tl is None:
            tl = token_len(d.text); d.meta["tok_len"] = tl
        if used + tl <= token_budget:
            acc.append(d.text); used += tl
        else:
            remain = token_budget - used
            if remain > 50:
                ids = llm_tokenizer(d.text, add_special_tokens=False)["input_ids"][:remain]
                acc.append(llm_tokenizer.decode(ids))
            break
    return "\n\n".join(acc)


In [13]:
# ===== 프롬프트 =====
def build_prompt(query: str, use_context: bool, context) -> Tuple[str, int]:
    """컨텍스트 유무에 따라 간단 프롬프트와 max_len을 반환"""
    # 객관식 질문
    if is_multiple_choice(query):
        question, options = extract_question_and_choices(query)
        if use_context:
            prompt = (
                "<<SYS>>\n당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요.\n<</SYS>>\n\n"
                "아래 컨텍스트를 우선 사용해 정확히 답하세요. 불충분하면 아는 범위에서만 간결히 답하세요.\n\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"=== 컨텍스트 ===\n{context}\n=== 끝 ===\n"
                f"질문: {query}"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
            )
            max_new = dynamic_max_new_tokens(query)
            max_len = 3072  # 입력 길이 상한도 줄여 토크나이즈 시간 단축
            return prompt, max_len
        else:
            prompt = (
                "<<SYS>>\n당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요.\n<</SYS>>\n\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {query}"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
            )
            max_new = dynamic_max_new_tokens(query)
            max_len = 2048
            return prompt, max_len
    # 주관식 질문
    else:
        if use_context:
            prompt = (
                "<<SYS>>\n당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요.\n<</SYS>>\n\n"
                "아래 컨텍스트를 우선 사용해 정확히 답하세요. 불충분하면 아는 범위에서만 간결히 답하세요.\n\n"
                f"=== 컨텍스트 ===\n{context}\n=== 끝 ===\n"
                """아래 질문에 대해 **사실에 근거한 간결한 답변**을 작성하세요. 

규칙:
1. 답변은 2~3문장 이내로 작성합니다. 장황한 서론, 결론 문구는 쓰지 않습니다.
2. 불확실할 경우 '알 수 없습니다'라고 답하고 생성을 종료합니다.
3. 특수문자 없이 오로지 한글과 숫자로만 대답합니다."""
                f"질문: {query}"
                "답변:"
            )
            max_new = dynamic_max_new_tokens(query)
            max_len = 3072  # 입력 길이 상한도 줄여 토크나이즈 시간 단축
            return prompt, max_len
        else:
            prompt = (
                "<<SYS>>\n당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요.\n<</SYS>>\n\n"
                """아래 질문에 대해 **사실에 근거한 간결한 답변**을 작성하세요. 

규칙:
1. 답변은 2~3문장 이내로 작성합니다. 장황한 서론, 결론 문구는 쓰지 않습니다.
2. 불확실할 경우 '알 수 없습니다'라고 답하고 생성을 종료합니다.
3. 특수문자 없이 오로지 한글과 숫자로만 대답합니다."""
                f"질문: {query}"
                "답변:"
            )
            max_new = dynamic_max_new_tokens(query)
            max_len = 2048
            return prompt, max_len


# ===== 생성 토큰 상한 =====
def dynamic_max_new_tokens(question: str) -> int:
    lines = [ln.strip() for ln in question.split("\n") if ln.strip()]
    opt_cnt = sum(bool(re.match(r"^\d+(\s|[.)])", ln)) for ln in lines)
    return 96 if opt_cnt >= 2 else 192

# ===== 메인: 다중 인덱스 기반 텍스트 생성 =====
def generate_answer_with_indices(query: str, indices: dict) -> str:
    """
    indices: build_indices() 반환 구조
    - 라우팅 → 해당 인덱스에서 top-k 검색(점수 포함)
    - 최고 유사도 임계치 미만이면 컨텍스트 없이 베이스모델 생성(Adaptive RAG)
    """
    # 0) 우선, 일반상식/비도메인은 곧장 베이스모델
    if not route_is_domain(query):
        prompt, max_len = build_prompt(query, use_context=False, context=None)
        inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_len, padding=False)
        inputs = {k: v.to(llm_model.device) for k, v in inputs.items()}
        with torch.inference_mode():
            out = llm_model.generate(**inputs,
                                     max_new_tokens=dynamic_max_new_tokens(query),
                                     do_sample=False, temperature=0.2,
                                     eos_token_id=llm_tokenizer.eos_token_id,
                                     pad_token_id=llm_tokenizer.pad_token_id)
        gen = out[0][inputs["input_ids"].shape[1]:]
        return llm_tokenizer.decode(gen, skip_special_tokens=True).strip()

    # 1) 인덱스 선택
    idx_entry = choose_index(indices, query)

    # 2) 인덱스가 없으면 베이스모델
    if idx_entry is None:
        prompt, max_len = build_prompt(query, use_context=False, context=None)
        inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_len, padding=False)
        inputs = {k: v.to(llm_model.device) for k, v in inputs.items()}
        with torch.inference_mode():
            out = llm_model.generate(**inputs,
                                     max_new_tokens=dynamic_max_new_tokens(query),
                                     do_sample=False, temperature=0.2,
                                     eos_token_id=llm_tokenizer.eos_token_id,
                                     pad_token_id=llm_tokenizer.pad_token_id)
        gen = out[0][inputs["input_ids"].shape[1]:]
        return llm_tokenizer.decode(gen, skip_special_tokens=True).strip()

    # 3) 선택 인덱스에서 검색 + 점수
    scored = faiss_search_with_scores_from_index(idx_entry, query, top_k=TOP_K)
    best_cos = max((s for _, s in scored), default=0.0)

    # 4) 임계치: 충분히 유사할 때만 컨텍스트 사용 (속도 최적화)
    THRESH = 0.70
    use_context = best_cos >= THRESH and len(scored) > 0

    context = pack_context([d for d, _ in scored], token_budget=CTX_TOKEN_BUDGET) if use_context else None
    prompt, max_len = build_prompt(query, use_context, context)

    # 5) LLM 생성
    inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_len, padding=False)
    inputs = {k: v.to(llm_model.device) for k, v in inputs.items()}
    with torch.inference_mode():
        out = llm_model.generate(
            **inputs,
            max_new_tokens=dynamic_max_new_tokens(query),
            do_sample=False,
            temperature=0.2,
            eos_token_id=llm_tokenizer.eos_token_id,
            pad_token_id=llm_tokenizer.pad_token_id,
        )
    gen = out[0][inputs["input_ids"].shape[1]:]
    return llm_tokenizer.decode(gen, skip_special_tokens=True).strip()


In [14]:
# ---------------- LLM 로드 & 생성 ----------------
llm_model = AutoModelForCausalLM.from_pretrained(
    LLM_ID,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    try:
        # llm_model.config.attn_implementation = "sdpa"
        llm_model.config.attn_implementation = "flash_attention_2"
        # llm_model.config.attn_implementation = "eager"
    except Exception:
        pass
llm_model.eval()
torch.set_grad_enabled(False)

config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [15]:
# 후처리 함수
def extract_answer_only(generated_text: str, original_question: str) -> str:
    """
    - "답변:" 이후 텍스트만 추출
    - 객관식 문제면: 정답 숫자만 추출 (실패 시 전체 텍스트 또는 기본값 반환)
    - 주관식 문제면: 전체 텍스트 그대로 반환
    - 공백 또는 빈 응답 방지: 최소 "미응답" 반환
    """
    # "답변:" 기준으로 텍스트 분리
    if "답변:" in generated_text:
        text = generated_text.split("답변:")[-1].strip()
    else:
        text = generated_text.strip()

    # 공백 또는 빈 문자열일 경우 기본값 지정
    if not text:
        return "미응답"

    # 객관식 여부 판단
    is_mc = is_multiple_choice(original_question)

    if is_mc:
        # 숫자만 추출
        match = re.match(r"\D*([1-9][0-9]?)", text)
        if match:
            return match.group(1)
        else:
            # 숫자 추출 실패 시 "0" 반환
            return "0"
    else:
        return text


In [16]:
preds = []

for q in tqdm(test['Question'], desc="Inference"):
    print("#################### Question ###########################")
    print(q)
    ans = generate_answer_with_indices(q, indices)
    pred_answer = extract_answer_only(ans, original_question=q)
    print("#################### Answer ###########################")
    print(pred_answer)
    preds.append(pred_answer)

Inference:   0%|          | 0/515 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


#################### Question ###########################
금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
1 소비자금융업
2 투자자문업
3 투자매매업
4 투자중개업
5 보험중개업


Inference:   0%|          | 1/515 [00:04<37:20,  4.36s/it]

#################### Answer ###########################
2
#################### Question ###########################
위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?
1 수행인력
2 위험 수용
3 위험 대응 전략 선정
4 대상
5 기간


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Inference:   0%|          | 2/515 [00:08<36:30,  4.27s/it]

...

#################### Answer ###########################
3
#################### Question ###########################
전자금융보안의 중요성이 부각되는 이유로 가장 적절한 것은?
1 IT 의존도의 감소로 인한 보안 필요성 감소
2 금융 분야의 자산과 국가 경제와의 직접적 연관성
3 금융산업의 규제 완화로 인한 보안 필요성 감소
4 금융거래의 비대면화로 인한 보안 위협 감소


Inference:  10%|█         | 54/515 [04:03<33:12,  4.32s/it]

#################### Answer ###########################
2
#################### Question ###########################
다음 중 분산 서비스 거부(DDoS) 공격에 대한 대응 방법으로 가장 적절한 것은?
1 웹 애플리케이션 방화벽을 통해 SQL 인젝션 공격을 차단한다.
2 데이터베이스의 암호화 수준을 높인다.
3 운영체제의 보안 패치를 최신으로 유지한다.
4 네트워크 트래픽을 모니터링하여 이상 징후를 탐지한다.
5 사용자 비밀번호를 주기적으로 변경한다.


Inference:  10%|█         | 54/515 [04:06<35:06,  4.57s/it]


KeyboardInterrupt: 

In [ ]:
sample_submission = pd.read_csv('../submission/sample_submission.csv')
sample_submission['Answer'] = preds
sample_submission.to_csv('../submission/gen_v4_submission.csv', index=False, encoding='utf-8-sig')